# 1. Settings

In [ ]:
#pip install numpy==1.21.6 pandas==2.0.3

In [ ]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import csv

import hanja
from hanja import hangul

# 2. Use hanja : 한자-한글 변환 라이브러리
- https://github.com/suminb/hanja
- site: https://www.pa.go.kr/research/contents/speech/index.jsp

### 1) Installation
-----

```python
   pip install hanja
```

### 2) Usage
------
필요한 모듈 import 하기
```python
   >>> import hanja
   >>> from hanja import hangul
```

한글 초성, 중성, 종성 분리
```python
   >>> hangul.separate('가')
   (0, 0, 0)
   >>> hangul.separate('까')
   (1, 0, 0)
   # 튜플(tuple)의 마지막 원소가 0이면 종성이 없는 글자라고 판단할 수 있다.
```
'ㅎ'은 19번째 자음, 'ㅏ'는 첫번째 모음, 'ㄴ'은 다섯번째 자음이라는 것을 알 수 있다.
```python
   >>> hangul.separate('한')
   (18, 0, 4)
```

초성, 중성, 종성을 조합하여 한 글자를 만듦
```python
   >>> hangul.build(0, 0, 0)
   '가'
```

주어진 글자가 한글인지의 여부를 판별
```python
   >>> hangul.is_hangul('가')
   True
   >>> hangul.is_hangul('a')
   False
```

한글로 된 부분과 한자로 된 부분을 분리
```python
   >>> '|'.join(hanja.split_hanja('大韓民國은 民主共和國이다.'))
   大韓民國|은 |民主共和國|이다.

   >>> [x for x in hanja.split_hanja('大韓民國은 民主共和國이다.')]
   ['大韓民國', '은 ', '民主共和國', '이다.']
# 리스트가 아닌 제네레이터(generator)를 반환한다.
```

주어진 글자가 한자인지의 여부를 판별
```python
   >>> hanja.is_hanja('韓')
   True

   >>> hanja.is_hanja('한')
   False
```

문장 변환
- 치환 모드 변환:
```python
   >>> hanja.translate('大韓民國은 民主共和國이다.', 'substitution')
   '대한민국은 민주공화국이다.'
```
- 혼용 모드 변환 (text):
```python
   >>> hanja.translate('大韓民國은 民主共和國이다.', 'combination-text')
   '大韓民國(대한민국)은 民主共和國(민주공화국)이다.'
```
- 혼용 모드 변환 version 2 (text):
```python
   >>> hanja.translate('大韓民國은 民主共和國이다.', 'combination-text-reversed')
   '대한민국(大韓民國)은 민주공화국(民主共和國)이다.'
```
- 혼용 모드 변환 (HTML):
```python
   >>> hanja.translate(u'大韓民國은 民主共和國이다.', 'combination-html')
   '<span class="hanja">大韓民國</span><span class="hangul">(대한민국)</span>은 <span class="hanja">民主共和國</span><span class="hangul">(민주공화국)</span>이다.'
```

# 3. Preprocessing

In [ ]:
name_file = ['LeeSeungMan', 'YoonBoSun', 'ParkJeongHee', 'ChoiGyuHa', 'JeonDuHwuan',
             'RohTaeWoo', 'KimYoungSam', 'KimDaeJung', 'RohMooHyun', 'LeeMyungBak',
             'MoonJaeIn', 'ParkGeunHye']

In [ ]:
# Function to remove non-Hangul characters within parentheses
def erase_hanja(text):
    result = []  # 결과를 저장할 빈 리스트 생성
    word_list = list(text)  # 입력 텍스트를 문자 리스트로 변환
    i = 0  # 인덱스 초기화
    while i < len(word_list):  # word_list의 길이만큼 반복
        if word_list[i] == '(':  # 현재 문자가 '('인지 확인
            temp = []  # 괄호 안의 한글을 임시로 저장할 리스트
            i += 1  # 인덱스를 다음 문자로 이동
            while i < len(word_list) and word_list[i] != ')':  # 닫는 괄호 ')'를 만날 때까지 반복
                if hangul.is_hangul(word_list[i]):  # 현재 문자가 한글인지 확인
                    temp.append(word_list[i])  # 한글이면 temp 리스트에 추가
                i += 1  # 인덱스를 다음 문자로 이동
            if i < len(word_list) and word_list[i] == ')':  # 닫는 괄호가 있는지 확인
                if temp:  # temp 리스트에 한글이 존재하면
                    result.append(''.join(temp))  # 한글만 있는 괄호를 result에 추가
                i += 1  # 닫는 괄호를 지나 인덱스를 다음 문자로 이동
            else:
                # 닫는 괄호가 없는 경우, 남은 텍스트를 무시하고 종료
                break
        else:  # 현재 문자가 '('가 아닌 경우
            result.append(word_list[i])  # result에 현재 문자 추가
            i += 1  # 인덱스를 다음 문자로 이동
    return ''.join(result)  # result 리스트를 문자열로 변환하여 반환

In [ ]:
# Load the CSV file
for _ in name_file:
  file_path = f'./dataset/raw_data/{_}.csv'
  df = pd.read_csv(file_path)
  data_list = df.to_dict(orient='records')

  ## RUN
  fixed_data = data_list

  for i in range(len(fixed_data)):
    #preprocessing title data
    title_d = fixed_data[i]['title']
    title_hanja_erased = erase_hanja(title_d)
    fixed_data[i]['title'] = hanja.translate(title_hanja_erased, 'substitution')
    #print(f"{i}. title: {fixed_data[i]['title']}")

    #preprocessing speech data
    speech_d = fixed_data[i]['speech']
    speech_hanja_erased = erase_hanja(speech_d)
    fixed_data[i]['speech'] = hanja.translate(speech_hanja_erased, 'substitution')
    #print(fixed_data[i]['speech'])


  ## To CSV
  folder_path = '.dataset/cleaned_data' 
  file_path = f'{folder_path}/cleaned_{_}.csv'

  # CSV 파일로 저장
  with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
      fieldnames = ["index", "title", "type", "date", "speaker", "speech"]
      writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

      writer.writeheader()
      for row in fixed_data:
          writer.writerow(row)

  print(f"SAVE COMPLETED: {file_path}")